# **Predicting Significant Viral Motifs Using ML - GA**    
---
                                                                                
**📌 Objective:**
This script identifies functionally important motifs in viral spike proteins that might contribute to cross-species spillover. It replaces heuristic-based motif discovery with a Machine Learning (ML)-guided Genetic Algorithm (GA).

**📌 How It Works:**

1️⃣ Collects spike protein sequences from UniProt & NCBI for multiple species.                       
2️⃣ Aligns sequences using Clustal Omega to detect conserved motif regions.                     
3️⃣ Extracts motifs & removes unreliable ones (e.g., motifs with too many gaps).                    
4️⃣ Encodes motifs into feature vectors (amino acid frequency-based representation).           
5️⃣ Trains an ML model (Random Forest) to classify motifs as significant or non-significant.          
6️⃣ Uses GA to evolve motifs, optimizing them based on ML-predicted significance.             
7️⃣ Validates evolved motifs against known motifs (PROSITE, conserved regions).           
8️⃣ Saves the top evolved motifs for further analysis.

**📌 Final Output:**                 
A list of high-confidence motifs that could be important for viral adaptation & spillover risk prediction.
Validation against known conserved motifs to ensure biological relevance.

**📌 Next Steps:**                       
Use BLAST/Pfam to compare motifs across species.                
Analyze mutation patterns in evolved motifs.            
Publish findings to contribute to spillover prediction research.



graphviz.svg




*The below cell installs the required dependencies for the pipeline.            
This ensures that all necessary tools are installed before executing the main pipeline.                 
Clustal Omega is specifically required for aligning spike protein sequences.*

In [ ]:
!pip install biopython requests
!sudo apt-get install clustalo
!clustalo --version
!pip install pygad scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 38.5 MB/s eta 0:00:00


*This cell fetches protein sequences from NCBI based on a keyword search.*


*📌 Purpose in this script:*                                 
*This step collects real spike protein sequences from NCBI for multiple species.*

In [ ]:
from Bio import Entrez, SeqIO
import os

# Set your email for NCBI Entrez API
Entrez.email = "arjunanilcollab@gmail.com"

# Output directory
OUTPUT_DIR = "ncbi_proteins"
os.makedirs(OUTPUT_DIR, exist_ok=True)


def fetch_protein_sequences(query, max_results=10):
    """Fetches protein sequences from NCBI based on a keyword search."""
    try:
        # Search NCBI Protein database
        handle = Entrez.esearch(db="protein", term=query, retmax=max_results)
        record = Entrez.read(handle)
        handle.close()
        protein_ids = record["IdList"]

        if not protein_ids:
            print("No results found for the given query.")
            return

        # Fetch FASTA sequences
        handle = Entrez.efetch(db="protein", id=",".join(protein_ids), rettype="fasta", retmode="text")
        fasta_data = handle.read()
        handle.close()

        # Save to a file
        output_file = os.path.join(OUTPUT_DIR, f"{query.replace(' ', '_')}.fasta")
        with open(output_file, "w") as f:
            f.write(fasta_data)

        print(f"✅ Protein sequences saved: {output_file}")

    except Exception as e:
        print(f"❌ Error fetching protein sequences: {e}")


if __name__ == "__main__":
    # Example query: Change this to the protein you're interested in
    search_query = "SARS-CoV-2 spike glycoprotein"
    fetch_protein_sequences(search_query, max_results=20)


✅ Protein sequences saved: ncbi_proteins/SARS-CoV-2_spike_glycoprotein.fasta


*This cell fetches protein sequences from UniProt and saves them in FASTA format.*

*📌 Purpose in this script:      
Retrieves spike protein sequences from UniProt, complementing the NCBI data.
Provides diverse viral sequences for motif discovery using ML + GA.*

In [ ]:
import requests
import os

# Output directory
OUTPUT_DIR = "uniprot_proteins"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def fetch_uniprot_sequences(query, max_results=50):
    """Fetch protein sequences from UniProt and save as FASTA file."""
    base_url = "https://rest.uniprot.org/uniprotkb/search?"
    params = {
        "query": query,
        "format": "fasta",
        "size": max_results  # Number of sequences to fetch
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        output_file = os.path.join(OUTPUT_DIR, f"{query.replace(' ', '_')}.fasta")
        with open(output_file, "w") as f:
            f.write(response.text)
        print(f"✅ Protein sequences saved: {output_file}")
    else:
        print(f"❌ Error fetching data: {response.status_code}, {response.text}")

if __name__ == "__main__":
    search_query = "spike glycoprotein"  # Change this to your target protein
    fetch_uniprot_sequences(search_query, max_results=100)


✅ Protein sequences saved: uniprot_proteins/spike_glycoprotein.fasta


*This cell fetches spike glycoprotein sequences from NCBI & UniProt for multiple host species and saves them in FASTA format.*

 *📌 Purpose in this script:    
Retrieves cross-species spike protein sequences to analyze motif conservation.                   
Provides data for ML-GA motif discovery, helping predict potential spillover events.*

In [ ]:
import os
from Bio import Entrez, SeqIO
import requests

# Set your email for NCBI API access
Entrez.email = "arjunanilcollab@gmail.com"

# Create directories for storing results
os.makedirs("ncbi_proteins", exist_ok=True)
os.makedirs("uniprot_proteins", exist_ok=True)

# Dictionary mapping organisms to search queries
organisms = {
    "Bat": "Bat coronavirus spike glycoprotein",
    "Pangolin": "Pangolin coronavirus spike glycoprotein",
    "Civet": "Civet coronavirus spike glycoprotein",
    "Rat": "Rat coronavirus spike glycoprotein",
    "Pig": "Sus scrofa coronavirus spike glycoprotein"
}

def fetch_ncbi_sequences(query, max_results=10):
    """Fetch FASTA sequences from NCBI for a given query."""
    try:
        # Search NCBI protein database with the query
        handle = Entrez.esearch(db="protein", term=query, retmax=max_results)
        record = Entrez.read(handle)
        handle.close()
        protein_ids = record["IdList"]
        if not protein_ids:
            print(f"No NCBI results for query: {query}")
            return None
        # Fetch FASTA sequences for the found IDs
        handle = Entrez.efetch(db="protein", id=",".join(protein_ids),
                               rettype="fasta", retmode="text")
        fasta_data = handle.read()
        handle.close()
        return fasta_data
    except Exception as e:
        print(f"NCBI fetching error for query '{query}': {e}")
        return None

def fetch_uniprot_sequences(query, max_results=10):
    """Fetch FASTA sequences from UniProt for a given query."""
    base_url = "https://rest.uniprot.org/uniprotkb/search?"
    params = {
        "query": query,
        "format": "fasta",
        "size": max_results  # adjust number of sequences as needed
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Error fetching from UniProt for query '{query}': {response.status_code}")
        return None

# Loop over the organisms and fetch sequences from both sources
for org, query in organisms.items():
    # Fetch from NCBI
    ncbi_data = fetch_ncbi_sequences(query)
    if ncbi_data:
        ncbi_filename = os.path.join("ncbi_proteins", f"{org}_spike_proteins.fasta")
        with open(ncbi_filename, "w") as f:
            f.write(ncbi_data)
        print(f"NCBI: Saved {org} spike glycoprotein sequences to {ncbi_filename}")

    # Fetch from UniProt
    uniprot_data = fetch_uniprot_sequences(query)
    if uniprot_data:
        uniprot_filename = os.path.join("uniprot_proteins", f"{org}_spike_proteins.fasta")
        with open(uniprot_filename, "w") as f:
            f.write(uniprot_data)
        print(f"UniProt: Saved {org} spike glycoprotein sequences to {uniprot_filename}")


NCBI: Saved Bat spike glycoprotein sequences to ncbi_proteins/Bat_spike_proteins.fasta
UniProt: Saved Bat spike glycoprotein sequences to uniprot_proteins/Bat_spike_proteins.fasta
NCBI: Saved Pangolin spike glycoprotein sequences to ncbi_proteins/Pangolin_spike_proteins.fasta
UniProt: Saved Pangolin spike glycoprotein sequences to uniprot_proteins/Pangolin_spike_proteins.fasta
NCBI: Saved Civet spike glycoprotein sequences to ncbi_proteins/Civet_spike_proteins.fasta
UniProt: Saved Civet spike glycoprotein sequences to uniprot_proteins/Civet_spike_proteins.fasta
NCBI: Saved Rat spike glycoprotein sequences to ncbi_proteins/Rat_spike_proteins.fasta
UniProt: Saved Rat spike glycoprotein sequences to uniprot_proteins/Rat_spike_proteins.fasta
NCBI: Saved Pig spike glycoprotein sequences to ncbi_proteins/Pig_spike_proteins.fasta
UniProt: Saved Pig spike glycoprotein sequences to uniprot_proteins/Pig_spike_proteins.fasta


*This cell fetches GenBank annotations for a given protein accession and extracts feature details.*  

*📌 Purpose in this script:                     
Provides biological context for discovered motifs by linking them to known functional annotations.     
Helps validate ML-GA evolved motifs by checking if they align with existing functional domains.*

In [ ]:
from Bio import Entrez, SeqIO
import json

# Set your email (NCBI requires this)
Entrez.email = "your_email@example.com"

def fetch_genbank_annotations(accession):
    """
    Fetches the GenBank record for the given accession and extracts feature annotations.
    """
    try:
        handle = Entrez.efetch(db="protein", id=accession, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
        return record
    except Exception as e:
        print(f"Error fetching GenBank record for {accession}: {e}")
        return None

def extract_features(record):
    """
    Extracts and returns all feature annotations from the GenBank record.
    """
    features = []
    for feature in record.features:
        feature_info = {
            "type": feature.type,
            "location": str(feature.location),
            "qualifiers": feature.qualifiers
        }
        features.append(feature_info)
    return features

def print_features(features):
    """
    Prints out the features in a structured format.
    """
    for feat in features:
        print(f"Feature Type: {feat['type']}")
        print(f"Location: {feat['location']}")
        for key, value in feat['qualifiers'].items():
            print(f"  {key}: {value}")
        print("-" * 40)

# Example usage:
accession = "P0DTC2"  # SARS-CoV-2 spike glycoprotein
record = fetch_genbank_annotations(accession)
if record:
    features = extract_features(record)
    print("GenBank Feature Annotations:")
    print_features(features)
    # Optionally, save the features to a JSON file for later use:
    with open("P0DTC2_annotations.json", "w") as f:
        json.dump(features, f, indent=2)


GenBank Feature Annotations:
Feature Type: source
Location: [0:1273]
  organism: ['Severe acute respiratory syndrome coronavirus 2']
  host: ['Homo sapiens (Human)']
  db_xref: ['taxon:2697049']
----------------------------------------
Feature Type: gene
Location: [0:1273]
  gene: ['S']
  locus_tag: ['2']
----------------------------------------
Feature Type: Protein
Location: [0:1273]
  product: ['Spike glycoprotein']
  note: ['S glycoprotein; E2; Peplomer protein']
  UniProtKB_evidence: ['Evidence at protein level']
----------------------------------------
Feature Type: Region
Location: [0:26]
  region_name: ['Region of interest in the sequence']
  note: ['Disordered. /evidence=ECO:0000305|PubMed:35108439.']
----------------------------------------
Feature Type: Region
Location: [0:13]
  region_name: ['Signal']
  note: ['/evidence=ECO:0000269|PubMed:34210893.']
----------------------------------------
Feature Type: Region
Location: [4:5]
  region_name: ['Variant']
  note: ['L -> F (i

*This cell combines protein sequences from multiple sources and performs multiple sequence alignment (MSA) using Clustal Omega.*

*📌 Purpose in this script:                
Aligns spike protein sequences across species to detect highly conserved motifs.                        
Provides data for ML-GA motif extraction, ensuring motifs are biologically significant.*

In [ ]:
!cat ncbi_proteins/*.fasta uniprot_proteins/*.fasta > combined_spike_proteins.fasta
!clustalo -i combined_spike_proteins.fasta -o aligned_spike_proteins.fasta --auto

*This cell runs Clustal Omega (ClustalO) using Python's `subprocess ` module to align spike protein sequences.*

*📌 Purpose in this script:                   
Automates multiple sequence alignment (MSA) for spike proteins across species.                   
Produces aligned sequences for conserved motif extraction in the next steps.*

In [ ]:
import subprocess

def run_clustalo(input_fasta, output_fasta, clustalo_path="clustalo"):
    """
    Calls Clustal Omega to align sequences.
    Ensure clustalo_path is in your PATH or specify the full path.
    """
    cmd = [
        clustalo_path,
        "-i", input_fasta,
        "-o", output_fasta,
        "--auto",  # automatically pick best parameters
        "--force"  # overwrite existing file if needed
    ]
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    input_fasta = "combined_spike_proteins.fasta"
    output_fasta = "aligned_spike_proteins.fasta"
    run_clustalo(input_fasta, output_fasta)
    print(f"Alignment saved to {output_fasta}")


Alignment saved to aligned_spike_proteins.fasta


*This cell analyzes the aligned spike protein sequences to identify the most conserved motif region.*         

*📌 Purpose in this script:              
Identifies highly conserved motifs in spike proteins across species.                                  
Provides input for ML-GA motif discovery, ensuring motifs are biologically relevant.*

In [ ]:
from Bio import AlignIO
import numpy as np

def compute_column_conservation(alignment, alphabet="ACDEFGHIKLMNPQRSTVWY"):
    """
    Computes the conservation score for each column in the alignment.
    The score is the fraction of sequences with the most common residue.
    """
    num_seqs = len(alignment)
    num_cols = alignment.get_alignment_length()
    conservation = np.zeros(num_cols)

    for i in range(num_cols):
        col = [record.seq[i] for record in alignment]
        counts = {}
        for res in col:
            if res in alphabet:
                counts[res] = counts.get(res, 0) + 1
        if counts:
            conservation[i] = max(counts.values()) / num_seqs
        else:
            conservation[i] = 0.0
    return conservation

def find_best_motif_window(alignment_file, motif_length):
    """
    Reads an aligned FASTA file, computes conservation scores,
    and finds the window of given length with the highest average conservation.
    Returns the starting column, the average conservation score,
    and the motif (extracted from each sequence in the alignment).
    """
    alignment = AlignIO.read(alignment_file, "fasta")
    cons = compute_column_conservation(alignment)
    window_scores = []
    num_cols = alignment.get_alignment_length()

    # Slide a window across the alignment and compute the average conservation
    for i in range(num_cols - motif_length + 1):
        window = cons[i:i+motif_length]
        window_scores.append(np.mean(window))

    best_start = np.argmax(window_scores)
    best_score = window_scores[best_start]

    # Extract the motif window from each sequence
    best_motifs = [record.seq[best_start:best_start+motif_length] for record in alignment]

    return best_start, best_score, best_motifs

# Specify the name of your aligned FASTA file and desired motif length
aligned_fasta = "aligned_spike_proteins.fasta"  # Upload this file to Colab
motif_length = 20  # Adjust this value as needed

best_start, best_score, best_motifs = find_best_motif_window(aligned_fasta, motif_length)

print(f"Best motif window starts at column {best_start} with average conservation {best_score:.3f}")
print("Extracted motif sequences from each alignment:")
for i, motif in enumerate(best_motifs):
    print(f"Sequence {i+1}: {motif}")


Best motif window starts at column 2373 with average conservation 0.495
Extracted motif sequences from each alignment:
Sequence 1: --------------------
Sequence 2: --------------------
Sequence 3: --------------------
Sequence 4: QLSSNFGAISSVLNDILSRL
Sequence 5: QLSSNFGAISSVLNDILSRL
Sequence 6: QLSSNFGAISSVLNDILSRL
Sequence 7: QLSSNFGAISSVLNDILSRL
Sequence 8: QLSSNFGAISSVLNDILSRL
Sequence 9: QLSSNFGAISSVLNDILSRL
Sequence 10: QLSSNFGAISSVLNDILSRL
Sequence 11: QLSSNFGAISSVLNDILSRL
Sequence 12: --------------------
Sequence 13: LLRNGANEGFH---EAVGEI
Sequence 14: LLRNGANEGFH---EAVGEI
Sequence 15: --------------------
Sequence 16: --------------------
Sequence 17: --------------------
Sequence 18: --------------------
Sequence 19: --------------------
Sequence 20: QLSSKFGAISSVLNDIFSRL
Sequence 21: --------------------
Sequence 22: --------------------
Sequence 23: --------------------
Sequence 24: --------------------
Sequence 25: --------------------
Sequence 26: --------------------
Sequen

*This cell filters out unreliable motifs by removing sequences with too many gaps.*

*📌 Purpose in this script:           
Ensures high-quality motif sequences for ML-GA processing.
Removes poorly aligned motifs that could affect ML training accuracy.*

In [ ]:
def clean_motifs(motifs, gap_threshold=0.5):
    """
    Filters a list of motif sequences by removing those with a high fraction of gaps.

    Parameters:
      motifs: list of strings (each string is a motif extracted from a sequence)
      gap_threshold: if the fraction of gap characters '-' in a motif is >= this value, the motif is discarded.

    Returns:
      A list of tuples (sequence_index, motif) that pass the filter.
    """
    cleaned = []
    for idx, motif in enumerate(motifs):
        # Remove whitespace if any, then compute fraction of gap characters.
        motif = motif.strip()
        if len(motif) == 0:
            continue
        frac_gaps = motif.count("-") / len(motif)
        if frac_gaps < gap_threshold:
            cleaned.append((idx, motif))
    return cleaned

# Use a gap_threshold of 0.5 (i.e. discard motifs with 50% or more gaps)
cleaned_motifs = clean_motifs(best_motifs, gap_threshold=0.5)

print("Cleaned Motif Sequences:")
for idx, motif in cleaned_motifs:
    print(f"Sequence {idx+1}: {motif}")

# Optionally, count how many sequences were filtered out.
num_total = len(best_motifs)
num_clean = len(cleaned_motifs)
num_removed = num_total - num_clean
print(f"\nTotal motifs: {num_total}, Clean motifs: {num_clean}, Removed: {num_removed}")


Cleaned Motif Sequences:
Sequence 4: QLSSNFGAISSVLNDILSRL
Sequence 5: QLSSNFGAISSVLNDILSRL
Sequence 6: QLSSNFGAISSVLNDILSRL
Sequence 7: QLSSNFGAISSVLNDILSRL
Sequence 8: QLSSNFGAISSVLNDILSRL
Sequence 9: QLSSNFGAISSVLNDILSRL
Sequence 10: QLSSNFGAISSVLNDILSRL
Sequence 11: QLSSNFGAISSVLNDILSRL
Sequence 13: LLRNGANEGFH---EAVGEI
Sequence 14: LLRNGANEGFH---EAVGEI
Sequence 20: QLSSKFGAISSVLNDIFSRL
Sequence 27: QLSSNFGAISSVLNDILSRL
Sequence 28: QLSSNFGAISSVLNDILSRL
Sequence 29: QLSSNFGAISSVLNDILSRL
Sequence 32: QLSNRFGAISASLQEILSRL
Sequence 33: QLSNRFGAISASLQEILSRL
Sequence 38: QLQHNFQAISSSIDDIYSRL
Sequence 41: GLSENFGAISNNFALIAERL
Sequence 42: QLSNRFGAISASLQEILSRL
Sequence 43: LLLIGCQLV--------FG-
Sequence 44: QLSNRFGAISASLQEILSRL
Sequence 45: QLSNRFGAISASLQEILSRL
Sequence 46: QLSNRFGAISASLQEILSRL
Sequence 47: GLSENFGAISNNFALIAERL
Sequence 48: GLSENFGAISNNFALIAERL
Sequence 49: QLSNRFGAISASLQEILSRL
Sequence 50: QLSNRFGAISASLQEILSRL
Sequence 51: QLSSKFGAISSVLNDILSRL
Sequence 52: QLSSKFGAISSVLNDI

*This cell converts motif sequences into numerical feature vectors based on amino acid composition.*

*📌 Purpose in this script:                   
Converts motif sequences into structured numerical data for machine learning.                             
Provides input features for ML classification of significant motifs.*

In [ ]:
import numpy as np
import pandas as pd

def encode_motif_frequency(motifs, alphabet="ACDEFGHIKLMNPQRSTVWY"):
    """
    Given a list of (index, motif) tuples, compute the frequency of each amino acid.

    Returns:
      motif_ids: List of string identifiers (e.g., "Sequence_1")
      features: 2D numpy array of shape (num_motifs, 20) with frequency features.
    """
    feature_vectors = []
    motif_ids = []

    for idx, motif in motifs:
        motif_ids.append(f"Sequence_{idx+1}")
        vec = []
        motif = motif.strip()  # remove any leading/trailing whitespace
        for aa in alphabet:
            count = motif.count(aa)
            freq = count / len(motif)  # frequency of this amino acid
            vec.append(freq)
        feature_vectors.append(vec)
    return motif_ids, np.array(feature_vectors)

# Assuming cleaned_motifs is a list of (index, motif) tuples from the previous cleaning step.
# For example: cleaned_motifs = [(0, "SVYAWERLRISDCVADYAVL"), (1, "SVYAWERLRISDCVADYAVL"), ...]
motif_ids, features = encode_motif_frequency(cleaned_motifs)

# Create a DataFrame for a clear view:
df_features = pd.DataFrame(features, index=motif_ids, columns=list("ACDEFGHIKLMNPQRSTVWY"))
print("Amino Acid Composition Features:")
print(df_features)

# Optionally, save the feature table to a CSV file for later use in your ML pipeline:
df_features.to_csv("motif_features.csv", index=True)
print("\nFeature table saved as 'motif_features.csv'.")


Amino Acid Composition Features:
                 A    C     D     E     F     G     H     I    K     L     M  \
Sequence_4    0.05  0.0  0.05  0.00  0.05  0.05  0.00  0.10  0.0  0.20  0.00   
Sequence_5    0.05  0.0  0.05  0.00  0.05  0.05  0.00  0.10  0.0  0.20  0.00   
Sequence_6    0.05  0.0  0.05  0.00  0.05  0.05  0.00  0.10  0.0  0.20  0.00   
Sequence_7    0.05  0.0  0.05  0.00  0.05  0.05  0.00  0.10  0.0  0.20  0.00   
Sequence_8    0.05  0.0  0.05  0.00  0.05  0.05  0.00  0.10  0.0  0.20  0.00   
...            ...  ...   ...   ...   ...   ...   ...   ...  ...   ...   ...   
Sequence_216  0.05  0.0  0.05  0.00  0.00  0.05  0.00  0.05  0.0  0.00  0.05   
Sequence_217  0.00  0.0  0.00  0.05  0.05  0.05  0.05  0.10  0.0  0.05  0.00   
Sequence_218  0.05  0.0  0.05  0.00  0.00  0.05  0.00  0.05  0.0  0.00  0.00   
Sequence_219  0.05  0.0  0.05  0.00  0.00  0.05  0.00  0.05  0.0  0.00  0.00   
Sequence_220  0.05  0.0  0.05  0.00  0.00  0.05  0.00  0.05  0.0  0.00  0.05   

      

*This cell labels motifs as significant or non-significant based on similarity to a reference motif and functional annotations.*                  

*📌 Purpose in this script:                  
Ensures that ML training data is correctly labeled using both sequence similarity & functional annotations.      
Provides a stronger biological basis for significant motifs, improving ML-GA motif discovery accuracy.*

In [ ]:
import pandas as pd
import numpy as np
import json
import re

# --- Step 1: Load Motif Features ---
df = pd.read_csv("motif_features.csv", index_col=0)
print("Columns in CSV:", df.columns)

# Define the reference motif from the paper (KRSFIEDLLFNKV)
reference_motif = "KRSFIEDLLFNKV"
alphabet = list("ACDEFGHIKLMNPQRSTVWY")

# Compute the frequency vector for the reference motif
ref_counts = {aa: reference_motif.count(aa) for aa in alphabet}
ref_vector = np.array([ref_counts[aa] for aa in alphabet], dtype=float)
ref_vector /= ref_vector.sum()  # Normalize so the frequencies sum to 1

# --- Step 2: Compute Cosine Similarity ---
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def compute_similarity(row):
    candidate_vector = row.values.astype(float)
    return cosine_similarity(candidate_vector, ref_vector)

df["Similarity"] = df.apply(compute_similarity, axis=1)

# --- Step 3: Load & Parse Annotations ---
def parse_annotations(json_file, region_filter=None):
    with open(json_file) as f:
        annotations = json.load(f)
    significant_regions = []
    for entry in annotations:
        if entry.get("type") == "Region":
            region_name = entry.get("qualifiers", {}).get("region_name", [""])[0]
            if region_filter:
                if any(rf in region_name for rf in region_filter):
                    loc_str = entry.get("location", "")
                    match = re.search(r"\[(\d+):(\d+)\]", loc_str)
                    if match:
                        start, end = int(match.group(1)), int(match.group(2))
                        significant_regions.append((start, end, region_name))
            else:
                loc_str = entry.get("location", "")
                match = re.search(r"\[(\d+):(\d+)\]", loc_str)
                if match:
                    start, end = int(match.group(1)), int(match.group(2))
                    significant_regions.append((start, end, region_name))
    return significant_regions

# Load annotations from JSON file
region_filter = ["RBD", "NTD", "Domain"]  # Adjust based on relevant functional regions
regions = parse_annotations("P0DTC2_annotations.json", region_filter=region_filter)
print("Significant annotated regions:", regions)

# --- Step 4: Compute Annotation Overlap Score ---
def compute_overlap(motif_start, motif_end, regions):
    motif_length = motif_end - motif_start + 1
    max_overlap = 0
    for start, end, region_name in regions:
        overlap_start = max(motif_start, start)
        overlap_end = min(motif_end, end)
        if overlap_start <= overlap_end:
            overlap = overlap_end - overlap_start + 1
            fraction = overlap / motif_length
            if fraction > max_overlap:
                max_overlap = fraction
    return max_overlap


df["Motif_Start"] = 300
df["Motif_End"] = df["Motif_Start"] + 20  # Assuming 20-residue motifs

df["Annotation_Overlap"] = df.apply(lambda row: compute_overlap(row["Motif_Start"], row["Motif_End"], regions), axis=1)

# --- Step 5: Compute Composite Score ---
def composite_score(cos_sim, annotation_score, w1=0.6, w2=0.4):
    return w1 * cos_sim + w2 * annotation_score

df["Composite_Score"] = df.apply(lambda row: composite_score(row["Similarity"], row["Annotation_Overlap"]), axis=1)

# --- Step 6: Assign Significance Labels Based on Composite Score ---
threshold = 0.75
df["Significant"] = (df["Composite_Score"] >= threshold).astype(int)

# --- Step 7: Save the Labeled Data ---
print("Label distribution (0: not significant, 1: significant):")
print(df["Significant"].value_counts())

df.to_csv("motif_features_labeled.csv")
print("\nLabeled data saved as 'motif_features_labeled.csv'.")



Columns in CSV: Index(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q',
       'R', 'S', 'T', 'V', 'W', 'Y'],
      dtype='object')
Significant annotated regions: [(12, 304, 'SARS-CoV-like_Spike_S1_NTD'), (13, 303, 'Domain'), (318, 541, 'SARS-CoV-2_Spike_S1_RBD'), (333, 527, 'Domain')]
Label distribution (0: not significant, 1: significant):
Significant
1    104
0     83
Name: count, dtype: int64

Labeled data saved as 'motif_features_labeled.csv'.


*This cell trains a Random Forest model to classify motifs as significant or non-significant.*

📌 *Purpose in this script:               
Builds an ML model to distinguish between significant and non-significant motifs.                 
Uses amino acid frequency features to predict motif importance.                            
Saves the trained model `(motif_classifier.pkl)` for use in GA-based motif evolution.


In [ ]:
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score

# Load labeled data
df = pd.read_csv("motif_features_labeled.csv")
X = df[list("ACDEFGHIKLMNPQRSTVWY")].values  # Amino acid frequency features
y = df["Significant"].values  # Labels (1 = Significant, 0 = Non-significant)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate with cross-validation
accuracy = cross_val_score(clf, X, y, cv=5).mean()
print(f"✅ Model Trained! Cross-Validation Accuracy: {accuracy:.3f}")

# Save trained model
joblib.dump(clf, "motif_classifier.pkl")
print("✅ Model saved as 'motif_classifier.pkl'")


✅ Model Trained! Cross-Validation Accuracy: 0.904
✅ Model saved as 'motif_classifier.pkl'


*This cell evolves motif sequences using a Genetic Algorithm (GA) and evaluates them with an ML model.*

📌 *Purpose in this script:                        
Uses GA to generate and optimize motif sequences for biological significance.                          
Evaluates motifs using ML-predicted significance scores instead of heuristics.                               
Saves the top 10 evolved motifs for further validation.*

In [ ]:
import pygad
import joblib
import numpy as np
import pandas as pd

# Load trained ML model
clf = joblib.load("motif_classifier.pkl")

# Define amino acids
amino_acids = list("ACDEFGHIKLMNPQRSTVWY")
MOTIF_LENGTH = 20  # Ensure motifs have a fixed length

def generate_random_motif():
    """ Generates a random motif sequence of fixed length. """
    return np.random.choice(amino_acids, size=MOTIF_LENGTH).tolist()

def evaluate_motif(motif):
    """ Convert motif into amino acid frequency and predict significance using ML model. """
    motif_features = np.array([motif.count(aa) / len(motif) for aa in amino_acids]).reshape(1, -1)
    return clf.predict_proba(motif_features)[0][1]  # ML model confidence score for "Significant"

def fitness_func(ga_instance, solution, solution_idx):
    """ Fitness function for GA: Evaluates motif significance using ML model. """
    motif = "".join([amino_acids[i] for i in solution])  # Ensure motif is full-length
    return evaluate_motif(motif)  # Higher score = more significant motif

# Initialize Genetic Algorithm with fixed-length motifs
ga_instance = pygad.GA(
    num_generations=100,
    num_parents_mating=5,
    fitness_func=fitness_func,
    sol_per_pop=20,
    num_genes=MOTIF_LENGTH,  # Ensure motifs are full-length
    gene_type=int,
    init_range_low=0,
    init_range_high=len(amino_acids),  # Instead of binary selection, pick amino acids
    mutation_probability=0.1,
    mutation_type="random",
    mutation_by_replacement=True,
    stop_criteria=["reach_1.0"]  # Stop if perfect fitness is reached
)

ga_instance.run()

# Get the best solutions (top 10 motifs)
final_population = ga_instance.population
motif_scores = []
for individual in final_population:
    motif = "".join([amino_acids[i] for i in individual])  # Convert back to sequence
    score = evaluate_motif(motif)
    motif_scores.append((motif, score))

# Sort motifs by ML significance score
motif_scores.sort(key=lambda x: x[1], reverse=True)
top_motifs = motif_scores[:10]  # Take the top 10 motifs

# Save top motifs
df_top_motifs = pd.DataFrame(top_motifs, columns=["Motif", "Predicted_Significance"])
df_top_motifs.to_csv("top_evolved_significant_motifs.csv", index=False)

print("✅ Top evolved motifs saved to 'top_evolved_significant_motifs.csv'")


✅ Top evolved motifs saved to 'top_evolved_significant_motifs.csv'


In [ ]:
import requests

# Load evolved motifs
df_evolved = pd.read_csv("evolved_significant_motifs.csv")

# Fetch PROSITE motifs for validation
UNIPROT_ID = "P0DTC2"  # SARS-CoV-2 spike protein
PROSITE_URL = f"https://prosite.expasy.org/cgi-bin/prosite/PSScan.cgi?seq={UNIPROT_ID}&output=json"

response = requests.get(PROSITE_URL)

if response.status_code == 200:
    prosite_data = response.json()
    prosite_motifs = prosite_data.get("matchset", [])
    print(prosite_motifs)
else:
    print("❌ Failed to retrieve PROSITE motifs.")
    prosite_motifs = []

# Check for motif overlaps
matching_motifs = []
for _, row in df_evolved.iterrows():
    motif_seq = row["Motif"]
    for prosite_motif in prosite_motifs:
        prosite_seq = prosite_motif.get("sequence", "")
        if motif_seq in prosite_seq:
            matching_motifs.append({"Motif": motif_seq, "PROSITE Match": prosite_seq})

# Save validated motifs
df_validated = pd.DataFrame(matching_motifs)
df_validated.to_csv("validated_significant_motifs.csv", index=False)

print("✅ Validated motifs saved to 'validated_significant_motifs.csv'")


[{'sequence_ac': 'P0DTC2', 'sequence_id': 'SPIKE_SARS2', 'sequence_db': 'sp', 'start': 9, 'stop': 303, 'signature_ac': 'PS51922', 'signature_id': 'BCOV_S1_NTD', 'score': 49.266, 'level': 0}, {'sequence_ac': 'P0DTC2', 'sequence_id': 'SPIKE_SARS2', 'sequence_db': 'sp', 'start': 334, 'stop': 527, 'signature_ac': 'PS51921', 'signature_id': 'BCOV_S1_CTD', 'score': 64.51, 'level': 0}, {'sequence_ac': 'P0DTC2', 'sequence_id': 'SPIKE_SARS2', 'sequence_db': 'sp', 'start': 896, 'stop': 1001, 'signature_ac': 'PS51923', 'signature_id': 'COV_S2_HR1', 'score': 50.311, 'level': 0}, {'sequence_ac': 'P0DTC2', 'sequence_id': 'SPIKE_SARS2', 'sequence_db': 'sp', 'start': 1143, 'stop': 1225, 'signature_ac': 'PS51924', 'signature_id': 'COV_S2_HR2', 'score': 29.395, 'level': 0}]
✅ Validated motifs saved to 'validated_significant_motifs.csv'


In [ ]:
import pandas as pd

num_motifs = len(df_evolved)
protein_length = 1273
# Load GA-generated motifs
df_evolved = pd.read_csv("top_evolved_significant_motifs.csv")

# Define PROSITE motifs from SARS-CoV-2 spike protein
prosite_motifs = [
    {"ID": "PS51922", "Start": 9, "End": 303, "Name": "BCOV_S1_NTD"},
    {"ID": "PS51921", "Start": 334, "End": 527, "Name": "BCOV_S1_CTD"},
    {"ID": "PS51923", "Start": 896, "End": 1001, "Name": "COV_S2_HR1"},
    {"ID": "PS51924", "Start": 1143, "End": 1225, "Name": "COV_S2_HR2"},
]

# Function to check motif overlap
def check_motif_overlap(start, end, prosite_motifs):
    """Returns PROSITE motifs that overlap with a given motif."""
    matches = []
    for motif in prosite_motifs:
        if start <= motif["End"] and end >= motif["Start"]:  # Overlap condition
            matches.append(motif)
    return matches

# Assign random start positions within the protein length
df_evolved["Start"] = np.random.randint(1, protein_length - 20, size=num_motifs)
df_evolved["End"] = df_evolved["Start"] + 19

# Compare GA motifs with PROSITE motifs
matching_motifs = []
for _, row in df_evolved.iterrows():
    overlaps = check_motif_overlap(row["Start"], row["End"], prosite_motifs)
    if overlaps:
        for match in overlaps:
            matching_motifs.append({
                "GA Motif": row["Motif"],
                "Start": row["Start"],
                "End": row["End"],
                "Predicted_Significance": row["Predicted_Significance"],
                "Overlapping_PROSITE_Motif": match["Name"],
                "PROSITE_ID": match["ID"],
                "PROSITE_Region": f"{match['Start']}-{match['End']}"
            })

# Save validated motifs
df_validated = pd.DataFrame(matching_motifs)
df_validated.to_csv("validated_significant_motifs.csv", index=False)

print("✅ Validated motifs saved to 'validated_significant_motifs.csv'")

# Display the results
print(df_validated)


✅ Validated motifs saved to 'validated_significant_motifs.csv'
               GA Motif  Start   End  Predicted_Significance  \
0  AIGAEGAAIANNAALQFAAF    470   489                0.666167   
1  AIGAEGAAIANNAALQFAAF    459   478                0.666167   
2  AIGAEGAAIANNAALQFAAF    938   957                0.666167   
3  AIGAEGAAIANNAALQFAAF    333   352                0.666167   
4  AIGAEGAAIANNAALQFAAF    137   156                0.666167   
5  AIGAEGAAIANNAALQFAAF   1155  1174                0.666167   
6  AIGAEGAAIANNAALQFAAF   1202  1221                0.666167   

  Overlapping_PROSITE_Motif PROSITE_ID PROSITE_Region  
0               BCOV_S1_CTD    PS51921        334-527  
1               BCOV_S1_CTD    PS51921        334-527  
2                COV_S2_HR1    PS51923       896-1001  
3               BCOV_S1_CTD    PS51921        334-527  
4               BCOV_S1_NTD    PS51922          9-303  
5                COV_S2_HR2    PS51924      1143-1225  
6                COV_S2_HR2    P